In [4]:
`gem install lingua`

"Successfully installed lingua-0.6.2\n1 gem installed\n"

In [1]:

require 'composable_operations'
require 'lemmatizer'
require 'lingua'
require 'pragmatic_segmenter'
require 'pragmatic_tokenizer'
require 'ruby-spacy'
require 'wordnet'


true

In [3]:
text = "Kernel panic, static, attic full of open source magic,
Command line acrobatics, shell syntax tactics,
Within the crypt of cryptic scripts, branches split and flip
Pull and merge tricks, where bugs get a fix and slip.

Relaxing on the echo of your echo, not so much unlike when the loops compute and dont flip on their condition in obstructive dispute. In the depths of code, a tale unfolds, gripping the command line, don't mind, fingers will turn cold.

Merging blurred lines of vision, refactoring enumerable recursions. with algorithmic eyes, paved with sleepless nights, to know an idea will have the mind make no other sense other than drive. concepts dissected, projected, connected"

# content = Lingua::EN::Readability.new(text)

"Kernel panic, static, attic full of open source magic,\nCommand line acrobatics, shell syntax tactics,\nWithin the crypt of cryptic scripts, branches split and flip\nPull and merge tricks, where bugs get a fix and slip.\n\nRelaxing on the echo of your echo, not so much unlike when the loops compute and dont flip on their condition in obstructive dispute. In the depths of code, a tale unfolds, gripping the command line, don't mind, fingers will turn cold.\n\nMerging blurred lines of vision, refactoring enumerable recursions. with algorithmic eyes, paved with sleepless nights, to know an idea will have the mind make no other sense other than drive. concepts dissected, projected, connected"

In [3]:
# p content.methods

In [ ]:
# get the POS of each word of the phrase processed with wordnet
# nlp = Spacy::Language.new("en_core_web_sm")


In [5]:
lexicon = WordNet::Lexicon.new 

#<WordNet::Lexicon:0x4790 sqlite:/home/jovyan/.local/share/gem/ruby/3.1.0/gems/wordnet-defaultdb-2.0.1/data/wordnet-defaultdb/wordnet31.sqlite>

In [25]:
synsets = lexicon.lookup_synsets('intellectual')

[#<WordNet::Synset:0x4df8 {109644715} 'intellect, intellectual' (noun): [noun.person] a person who uses the mind creatively>, #<WordNet::Synset:0x4e20 {301335338} 'intellectual' (adjective): [adj.all] appealing to or using the intellect>, #<WordNet::Synset:0x4e48 {301784888} 'intellectual, noetic, rational' (adjective satellite): [adj.all] of or associated with or requiring the use of the mind>, #<WordNet::Synset:0x4e70 {301933854} 'cerebral, intellectual' (adjective): [adj.all] involving intelligence rather than emotions or instinct>]

In [19]:
keywords = []
synsets.each do |synset|
    keywords += synset.words.map(&:lemma) # Synonyms

    synset.hyponyms.each do |hyponym_set|
        keywords += hyponym_set.words.map(&:lemma)
    end
end

keywords.uniq! # Remove duplicates


["cerebral", "intellectual"]

In [20]:
p keywords

["cerebral", "intellectual"]


["cerebral", "intellectual"]

In [46]:
word1_syns = lexicon.lookup_synsets( 'intellectual', :adjective )
word2_syns = lexicon.lookup_synsets( 'individual', :adjective )

[#<WordNet::Synset:0x5460 {300495505} 'individual, single' (adjective): [adj.all] being or characteristic of a single thing or person>]

In [47]:
# Use the analyzer to traverse hypernyms of the synset, adding a string for each
# one with indentation for the level
word1_syns.each do |syn|
	puts ">>> Searching with #{syn.wordlist.join(', ')} as the origin."

	word2_syns.each do |syn2|
		puts "  Comparing #{syn2.wordlist.join(', ')} to the origin."

		# The intersection of the two synsets is the most-specific hypernym they
		# share in common.
		common_syn = (syn | syn2)
        p syn
        #p syn2.methods
        #p syn2.hypernyms
        #p syn2.hyponyms
        p syn.similar_words
        p syn2.similar_words
		# Skip common synsets that are too abstract
		if common_syn && common_syn.lexical_domain != 'noun.tops'
			puts syn, syn2, '  ' + common_syn.to_s, ''
		else
			puts "    No synsets in common."
		end
	end

	puts "  done with #{syn.wordlist}"
end

>>> Searching with intellectual as the origin.
  Comparing individual, single to the origin.
#<WordNet::Synset:0x5488 {301335338} 'intellectual' (adjective): [adj.all] appealing to or using the intellect>
[#<WordNet::Synset:0x54b0 {301335859} 'highbrow, highbrowed' (adjective satellite): [adj.all] highly cultured or educated>, #<WordNet::Synset:0x54d8 {301336070} 'rational' (adjective satellite): [adj.all] having its source in or being guided by the intellect (as distinguished from experience or emotion)>, #<WordNet::Synset:0x5500 {301336283} 'reflective' (adjective satellite): [adj.all] devoted to matters of the mind>, #<WordNet::Synset:0x5528 {301336429} 'good, serious' (adjective satellite): [adj.all] appealing to the mind>, #<WordNet::Synset:0x5550 {301336561} 'sophisticated' (adjective satellite): [adj.all] intellectually appealing>]
[#<WordNet::Synset:0x5578 {300495865} 'idiosyncratic' (adjective satellite): [adj.all] peculiar to the individual>, #<WordNet::Synset:0x55a0 {3004960

[#<WordNet::Synset:0x5488 {301335338} 'intellectual' (adjective): [adj.all] appealing to or using the intellect>, #<WordNet::Synset:0x5640 {301933854} 'cerebral, intellectual' (adjective): [adj.all] involving intelligence rather than emotions or instinct>]

In [33]:

class WordSenses < ComposableOperations::ComposedOperation
  processes :word

  before do
    @lexicon = WordNet::Lexicon.new
  end

  def hypernyms
    @origins.each_with_index do |syn, _idx|
      @hypernyms = []
      # Traverse the hypernyms
      syn.traverse(:hypernyms).with_depth.each do |hyper_syn, depth|
        indent = "  " * depth
        @hypernyms << format("%s%s", indent, hyper_syn)
      end
      puts "Tree has #{@hypernyms.length} synsets."
    end
    @hypernyms
  end

  def execute
    @origins = @lexicon.lookup_synsets(word, :noun)
    hypernyms
  end
end


class Lemma < ComposableOperations::Operation
  processes :word

  before do
    @lem = Lemmatizer.new
  end

  def execute
    @lem.lemma(word)
  end
end


:execute

In [34]:
WordSenses.perform("intellectual")

Tree has 10 synsets.


["  individual, mortal, person, somebody, someone, soul (noun): [noun.tops] a human being (hypernym: 2, hyponym: 412, member meronym: 1, part holonym: 2)", "    being, organism (noun): [noun.tops] a living thing that has (or can develop) the ability to act or function independently (domain member category: 9, hypernym: 1, hyponym: 48, part holonym: 2, substance holonym: 1)", "      animate thing, living thing (noun): [noun.tops] a living (or once living) entity (domain member category: 2, hypernym: 1, hyponym: 4)", "        unit, whole (noun): [noun.tops] an assemblage of parts that is regarded as a single entity (hypernym: 1, hyponym: 7, part holonym: 2)", "          object, physical object (noun): [noun.tops] a tangible and visible entity; an entity that can cast a shadow (hypernym: 1, hyponym: 37)", "            physical entity (noun): [noun.tops] an entity that has physical existence (hypernym: 1, hyponym: 6)", "              entity (noun): [noun.tops] that which is perceived or kn

In [48]:
Lemma.perform("cerebral")

"cerebral"

In [56]:
require 'wordnet'

module MalapropismDetector
  class WordNetManager
    def initialize
      @wordnet = WordNet::Lexicon.new
    end

    def get_synsets(word)
      @wordnet.lookup_synsets(word)
    end
  end

  class MalapropismChecker
    def initialize(wordnet_manager)
      @wordnet_manager = wordnet_manager
    end

    def check_malapropism(word1, word2)
      synsets1 = @wordnet_manager.get_synsets(word1)
      synsets2 = @wordnet_manager.get_synsets(word2)

      return false if synsets1.empty? || synsets2.empty?
    
        p synsets1

      #synsets1.any? { |synset1| synsets2.any? { |synset2| synset1.lemma_names.include?(synset2.lemma_names.first) } }
    end
  end
end

# Usage example:

wordnet_manager = MalapropismDetector::WordNetManager.new
malapropism_checker = MalapropismDetector::MalapropismChecker.new(wordnet_manager)

puts malapropism_checker.check_malapropism("complement", "compliment") # true
puts malapropism_checker.check_malapropism("word", "sentence") # false

[#<WordNet::Synset:0x5d48 {105116811} 'accompaniment, complement' (noun): [noun.attribute] something added to complete or embellish or make perfect>, #<WordNet::Synset:0x5d70 {105704262} 'complement' (noun): [noun.cognition] either of two parts that mutually complete each other>, #<WordNet::Synset:0x5d98 {106326631} 'complement' (noun): [noun.communication] a word or phrase used to complete a grammatical construction>, #<WordNet::Synset:0x5dc0 {108235133} 'complement, full complement' (noun): [noun.group] number needed to make up a whole force>, #<WordNet::Synset:0x5de8 {113613201} 'complement' (noun): [noun.quantity] a complete number or quantity>, #<WordNet::Synset:0x5e10 {114760380} 'complement' (noun): [noun.substance] one of a series of enzymes in the blood serum that are part of the immune response>, #<WordNet::Synset:0x5e38 {200456874} 'complement' (verb): [verb.change] make complete or perfect; supply what is wanting or form the complement to>]
accompaniment, complement (noun):

In [62]:

module MalapropismDetector
  class Lemmatizer
    def initialize
      @lemmatizer = Lemmatizer.new
    end

    def lemma(word, part_of_speech = :verb)
      @lemmatizer.lemma(word, part_of_speech)
    end
  end

#   class MalapropismChecker
#     def initialize(lemmatizer)
#       @lemmatizer = lemmatizer
#     end

#     def check_malapropism(word1, word2)
#       lemma1 = @lemmatizer.lemma(word1)
#       lemma2 = @lemmatizer.lemma(word2)

#       return false if lemma1.nil? || lemma2.nil?

#       lemma1.to_s.downcase == lemma2.to_s.downcase
#     end
#   end
end

# Usage example:

l = Lemmatizer.new
# malapropism_checker = MalapropismDetector::MalapropismChecker.new(lemmatizer)

# puts malapropism_checker.check_malapropism("complement", "compliment") # true
# puts malapropism_checker.check_malapropism("word", "sentence") # false

#<Lemmatizer::Lemmatizer:0x000077a64c7864e0>

In [70]:
one = l.lemma("word")
two = l.lemma("sentence")

"sentence"

In [71]:
one == two

false